# Replay Script

This script will provide the ability to replay what has happened in the market, with the user being able to specify the time they wish to view. 

- Future iterations will allow for the simulation to be re-ran in real time, or possibly slowing down or speeding up the replay system.

In [11]:
# import libraries
%matplotlib agg
import time
import os
import io
import warnings
import mplfinance as mpf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from prettytable import PrettyTable
from ipywidgets import AppLayout


# Set the pandas option to opt into the future behavior, and suppress any SettingWithCopyWarning and FutureWarning messages
pd.set_option('future.no_silent_downcasting', True)
pd.options.mode.chained_assignment = None 
warnings.simplefilter(action='ignore', category=FutureWarning)

### WIDGET SETUP ####
file_name_input = widgets.Text(
    description='Enter File Name:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(height='50px')
)
file_submit_button = widgets.ToggleButton(
    value = False,
    description='Click to submit.',
    disabled = False,
    icon='check',
    layout=widgets.Layout(height='50px')
)
file_submit = widgets.HTML(layout=widgets.Layout(height='50px'))

time_int = widgets.IntText(
    value=0,
    min=0,
    step=1,
    description='Time After Start (Seconds):',
    style={'description_width': 'initial'}
)
time_slider_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    continuous_update=True,
    layout=widgets.Layout(padding='30px', align_content='center', justify_content='center')
)
time_play = widgets.Play(
    value=0,
    min=0,
    max=0,
    interval=100,
    description="Press Play",
    disabled = False
)
# Custom CSS to adjust the width of the individual buttons
style = """
<style>
.custom-toggle-buttons .widget-toggle-button {
    width: 50px;  
}
</style>
"""
time_speed = widgets.ToggleButtons(
    options=[('x0.5', 0.5), ('x1', 1), ('x2', 2), ('x4', 4), ('x8', 8), ('x16', 16)],
    value=1,
    disabled=False,
    style={'button_wdith': '50px'}
    #layout=widgets.Layout(width='150px')
)
time_speed.add_class('custom-toggle-buttons')

time_direction = widgets.ToggleButtons(
    options=[('<<<', -1), ('>>>', 1)],
    value = 1,
    disabled=False,
    style={'button_wdith': '50px'}
)
time_direction.add_class('custom-toggle-buttons')

widgets.jslink((time_slider_widget, 'value'), (time_play, 'value'))
widgets.jslink((time_slider_widget, 'value'), (time_int, 'value'))

time_render = widgets.HTML()

# Ordering the widges into the header
top_grid = widgets.GridspecLayout(5,1)
file_handler = widgets.HBox([file_name_input, file_submit_button])
time_section = widgets.HBox([time_int, time_render])
player = widgets.HBox([time_play, time_slider_widget])
speed_section = widgets.HBox([time_direction, time_speed])
top_grid[0,0] = file_handler
top_grid[1,0] = file_submit
top_grid[2,0] = time_section
top_grid[3,0] = player
top_grid[4,0] = speed_section

b_orderbook_title = widgets.HTML(value="""
                                 <html>
                                    <h3>
                                        <i>Buy Orderbook</i>
                                    </h3>
                                </html> """)
s_orderbook_title = widgets.HTML(value="""
                                 <html>
                                    <h3>
                                        <i>Sell Orderbook</i>
                                    </h3>
                                </html> """)
b_orderbook_widget = widgets.HTML()
s_orderbook_widget = widgets.HTML()
main_html = widgets.HTML()
chart_widget = widgets.Image(format='png')

centre_grid=widgets.GridspecLayout(2,3)
centre_grid[0,0] = b_orderbook_title
centre_grid[1,0] = b_orderbook_widget
centre_grid[0,1] = main_html
centre_grid[1,1] = chart_widget
centre_grid[0,2] = s_orderbook_title
centre_grid[1,2] = s_orderbook_widget
centre_grid.layout.grid_template_rows = '100px auto'
centre_grid.layout.grid_template_columns = '200px 800px 200px'



### Functions to handle and update the front end ###
# Creating an "output" folder which is used to read in the file, and creating the file input output
def file_read(*args): 
    global orderbook_log, transaction_log, start_time
    try:
        output_path = 'output'
        file_name = file_name_input.value
        final_output_path = os.path.join(output_path, file_name)

        # reading the input file, passing the first column as the index
        orderbook_log = pd.read_excel(final_output_path, sheet_name='Orderbook_Log', index_col=0)
        transaction_log = pd.read_excel(final_output_path, sheet_name='Transaction_Log', index_col=0)

        # update front end to be success
        file_submit_button.button_style = 'success'
        file_submit.value = """
        <html>
            <body>
                <b>
                {file_name} was successfully read, and ready to be visualised!
                </b>
            </body>
        </html>
        """.format(file_name= file_name)

        # update the ranges for the time slider
        start_time = orderbook_log.iloc[0]["Timestamp"].timestamp()
        end_time = orderbook_log.iloc[-1]["Timestamp"].timestamp()
        end_delta = end_time - start_time
        time_slider_widget.max = end_delta
        time_play.max = end_delta

    except:
        # update front end to be failure
        file_submit_button.button_style = 'danger'
        file_submit.value = """
        <html>
            <body>
                <b>
                {file_name} could not be read. Please check the file name.
                </b>
            </body>
        </html>
        """.format(file_name= file_name)
file_submit_button.observe(file_read, 'value')


# function that handles the creation of orderbook vis
def orderbook_vis (orderbook, side, html_flag):
    agg_orderbook = pd.DataFrame(columns=["Price", "Quantity"])
    agg_orderbook = orderbook.groupby("Price").agg({"Quantity" : 'sum'}).reset_index()
    agg_orderbook.columns = ['Price', 'Total Quantity']
    
    if html_flag == True:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        if side == "b":
            agg_orderbook.sort_values(by=['Price'], ascending=False, inplace=True) 
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
        vis = vis.get_html_string()
    elif html_flag == False:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
    return vis

# function to update the buy_orderbook based on the time selected by the user
def update_orderbook (time):
    global orderbook_log
    orderbook = pd.DataFrame(columns=["Order_ID", "Trader_ID", "Timestamp", "Quantity", "Price", "Side"])
    for index, order in orderbook_log.iterrows():
        # look for any open orders or any orders that were open at that point in time
        if (order["Status"] == "Open" and order["Timestamp"] <= time) or (order["Status"] != "Open" and (order["Timestamp"] <= time and order["Update_Timestamp"] > time)):        
            new_order = {"Order_ID": order["Order_ID"], "Trader_ID" : order["Trader_ID"], "Timestamp" : order["Timestamp"], "Quantity" : order["Quantity"], "Price" : order["Price"], "Side": order["Side"]}
            to_orderbook = pd.Series(new_order)
            orderbook = pd.concat([orderbook, to_orderbook.to_frame().T], ignore_index=True)
    # Splitting orderbook out into separate buy and sell sides
    buy_condition = orderbook["Side"] == "Buy"
    buy_orderbook = orderbook[buy_condition]  
    sell_orderbook = orderbook[~buy_condition]
    buy_orderbook.drop(["Side"], axis=1, inplace=True)
    sell_orderbook.drop(["Side"], axis=1, inplace=True)

    # organising orderbooks, and converting them into html format 
    buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
    b_orderbook_html = orderbook_vis(buy_orderbook, "b", html_flag=True)

    sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
    s_orderbook_html = orderbook_vis(sell_orderbook, "s", html_flag=True)

    return b_orderbook_html, s_orderbook_html

# fuction to update the candlestick chart
def update_transaction (time, time_elapsed):
    global transaction_log

    # pulling the transactions that fall within the time window, and ordering them into candles
    vis_transaction = pd.DataFrame(columns=["Timestamp", "Buy_Side_Order_ID", "Buy_Side_Trader_ID", "Sell_Side_Order_ID", "Sell_Side_Trader_ID", "Quantity", "Price", "Aggressor", "Aggressor_ID"])  
    for index, transaction in transaction_log.iterrows():
        if transaction["Timestamp"] <= time:
            vis_transaction = pd.concat([vis_transaction, transaction.to_frame().T], ignore_index=True)
    vis_transaction["Timestamp"] = pd.to_datetime(vis_transaction["Timestamp"])
    vis_transaction.sort_values(by=["Timestamp"], ascending=[True], inplace=True)
    market_price = vis_transaction["Price"].iloc[-1]

    if time_elapsed < 180:
        candle_length = 15 
    elif time_elapsed >= 180 and time_elapsed < 300:
        candle_length = 30
    elif time_elapsed >= 300 and time_elapsed < 900:
        candle_length = 60
    elif time_elapsed >= 900:
        candle_length = 120

    str_candle_length = "".join((str(candle_length),'s'))
    vis_transaction["timestamp"] = vis_transaction['Timestamp'].dt.floor(str_candle_length)
    vis_transaction = vis_transaction.groupby("timestamp").agg({
        "Price" : ['first', 'max', 'min','last'],
        "Quantity" : 'sum'
    })
    vis_transaction.columns = ['open', 'high', 'low', 'close', 'volume']
    columns_to_convert = ['open', 'high', 'low', 'close']
    vis_transaction[columns_to_convert] = vis_transaction[columns_to_convert].round(2).astype(float)
    vis_transaction["volume"] = vis_transaction["volume"].astype(int)

    # chart settings
    chart_colors = mpf.make_marketcolors(
        up='green',
        down='red'
    )
    chart_style = mpf.make_mpf_style(
        marketcolors = chart_colors,
        gridcolor='lightgray',        
        gridstyle='--'
    )

    fig, ax = mpf.plot(vis_transaction, type='candle', style=chart_style, volume=True, returnfig=True)   
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    chart_widget_html = buf.getvalue()


    main_html.value = """
        <html>
            <body>
                <h1 align="center">Market Price: {market_price}</h1>
                <p align="center">
                <i> Time: {time}. &emsp;&emsp; Current playback speed is: x{speed}</i>
                </p>
            </body>
        </html>
        """.format(market_price=market_price,
                   time=time,
                   speed=time_speed.value)
    
    return chart_widget_html

# fucntion to set the time direction
def direction_set(change):
    global direction, speed
    direction = change['new']
    time_play.step = direction
    time_play.interval = 100 // speed
time_direction.observe(direction_set, names='value')

# function to scale playback speed based on the time_speed widget selection
def replay_speed(change):
    global direction, speed
    speed = change['new']
    time_play.step = direction
    time_play.interval = 100 // speed
time_speed.observe(replay_speed, names='value')


# function to continuously render when the play button is pressed
def continuous_updates(vaue):
    time_value = time_int.value
    time_elapsed = (start_time + time_value) - 3600
    datetime_point = dt.datetime.fromtimestamp(time_elapsed)
    b_orderbook_widget.value, s_orderbook_widget.value = update_orderbook(datetime_point)
    chart_widget.value = update_transaction(datetime_point, time_elapsed)

def on_value_change(change):
    continuous_updates(change['new'])

def on_play_button(change):
    if change['new']:
        time_render.value = """
                <html>
                    <body>
                        <b>
                        Playback is running.
                        </b>
                    </body>
                </html>
                """
    else:
        time_render.value = """
                <html>
                    <body>
                        <b>
                        Playback is paused.
                        </b>
                    </body>
                </html>
                """

time_play.observe(on_value_change, names='value')
time_play.observe(on_play_button, names='playing')

app_layout = AppLayout(header=top_grid,center=centre_grid)
display(widgets.HTML(style)) # injecting custom CSS styling for the speed buttons
display(app_layout)



HTML(value='\n<style>\n.custom-toggle-buttons .widget-toggle-button {\n    width: 50px;  \n}\n</style>\n')

AppLayout(children=(GridspecLayout(children=(HBox(children=(Text(value='', description='Enter File Name:', lay…

NameError: name 'speed' is not defined

NameError: name 'speed' is not defined

IndexError: single positional indexer is out-of-bounds

IndexError: single positional indexer is out-of-bounds